### Semana 1: Setup y Pipeline Básico

**Objetivos:**
- Tener el entorno configurado
- Pipeline de ingesta funcionando
- Primeras pruebas con ChromaDB

**Tareas:**
- [ ] Crear repositorio en GitHub
- [ ] Configurar entorno virtual y crear requirements.txt
- [ ] Recolectar/generar corpus inicial (5-10 documentos)
- [ ] Instalar y configurar LangChain
- [ ] Implementar carga de documentos (TextLoader, PyPDFLoader, etc.)
- [ ] Implementar chunking con RecursiveCharacterTextSplitter
- [ ] Configurar modelo de embeddings
- [ ] Crear base de datos vectorial con ChromaDB
- [ ] Probar ingesta: documentos → chunks → embeddings → ChromaDB
- [ ] Verificar persistencia (que los datos se guarden)


In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/michellzambranohereira/PLN/refs/heads/main/TP%20Final%20Integrador/dataset/rese%C3%B1as.csv"
df = pd.read_csv(url)

df.head()

,titulo_pelicula,reseña,sentimiento
0,Harry Potter,Cada libro de Harry Potter ofrece una evolució...,positivo
1,Harry Potter,La magia en Harry Potter está tan bien constru...,positivo
2,Harry Potter,La magia en Harry Potter está tan bien constru...,positivo
3,Harry Potter,La saga de Harry Potter me sorprendió por su m...,positivo
4,Harry Potter,Sentí que la historia de Harry Potter avanzaba...,negativo


In [ ]:
!pip install langchain chromadb sentence-transformers langchain-community


In [8]:
import pandas as pd
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import warnings
warnings.filterwarnings("ignore")

# URL del dataset
url = "https://raw.githubusercontent.com/michellzambranohereira/PLN/refs/heads/main/TP%20Final%20Integrador/dataset/rese%C3%B1as.csv"

# 1. Cargar el CSV desde la URL
df = pd.read_csv(url)

# 2. Convertir cada fila en Document
documentos = [
    Document(
        page_content=row["reseña"],
        metadata={
            "titulo_pelicula": row["titulo_pelicula"],
            "sentimiento": row["sentimiento"]
        }
    )
    for _, row in df.iterrows()
]

print(f"Corpus preparado con {len(documentos)} reseñas")

# 3. Dividir en chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
chunks = splitter.split_documents(documentos)

print(f"Corpus dividido en {len(chunks)} chunks")
# 4. Crear embeddings
embeddings = HuggingFaceEmbeddings()

# 5. Guardar en ChromaDB
db = Chroma.from_documents(chunks, embeddings, persist_directory="./reseñas_db")

print(f"Documentos almacenados en ChromaDB: {db._collection.count()}")

Corpus preparado con 100 reseñas
Corpus dividido en 100 chunks
Documentos almacenados en ChromaDB: 200


### Semana 2: Integración del LLM y Retrieval

**Objetivos:**
- Integrar modelo de lenguaje elegido
- Implementar recuperación semántica
- Cadena RAG funcionando end-to-end

**Tareas:**
- [ ] Configurar LLM elegido (Gemini/Ollama/HuggingFace)
- [ ] Crear retriever desde ChromaDB
- [ ] Configurar parámetros de búsqueda (top-k, similarity threshold)
- [ ] Implementar cadena RetrievalQA con LangChain
- [ ] Diseñar prompt template para el contexto
- [ ] Probar recuperación + generación con consultas manuales
- [ ] Implementar return_source_documents para citación
- [ ] Manejar casos donde no hay documentos relevantes
- [ ] Ajustar parámetros (chunk_size, k, temperature, etc.)

**Código conceptual:**
```python
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama  # o el que uses

# Cargar LLM
llm = Ollama(model="llama3.1:8b")

# Cargar vector store existente
db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

# Crear retriever
retriever = db.as_retriever(search_kwargs={"k": 3})

# Crear cadena RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# Probar
resultado = qa_chain({"query": "Tu pregunta de prueba"})
print(resultado["result"])
print("Fuentes:", [doc.metadata for doc in resultado["source_documents"]])
```

**Checkpoint de fin de semana 2:**
- ¿El sistema responde preguntas basándose en tus documentos?
- ¿Las respuestas tienen sentido y están fundamentadas?
- ¿Podés ver qué documentos se usaron?
- ¿Probaste con diferentes tipos de consultas?

---

In [ ]:
!pip install -q langchain chromadb sentence-transformers transformers accelerate langchain-community

In [9]:
import pandas as pd
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# --- 1) Cargar CSV y preparar documentos ---
# URL del dataset
url = "https://raw.githubusercontent.com/michellzambranohereira/PLN/refs/heads/main/TP%20Final%20Integrador/dataset/rese%C3%B1as.csv"
df = pd.read_csv(url)

documentos = [
    Document(
        page_content=row["reseña"],
        metadata={
            "titulo_pelicula": row["titulo_pelicula"],
            "sentimiento": row["sentimiento"]
        }
    )
    for _, row in df.iterrows()
]

print(f"Corpus preparado con {len(documentos)} reseñas")

# --- 2) Chunking ajustado (cada reseña completa) ---
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = splitter.split_documents(documentos)
print(f"Corpus dividido en {len(chunks)} chunks")

# --- 3) Embeddings más robustos ---
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# --- 4) Vector store en memoria (evita error de permisos en Colab) ---
db = Chroma.from_documents(
    chunks,
    embeddings,
    collection_name="resenas_collection_384"  # nombre válido
)
retriever = db.as_retriever(search_kwargs={"k":5})


# --- 5) LLM local con Transformers (Flan-T5 base) ---
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
hf_pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer,
                   max_new_tokens=256, temperature=0)
llm = HuggingFacePipeline(pipeline=hf_pipe)

# --- 6) Cadena RAG con map_reduce ---
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",   # <- mejora precisión
    retriever=retriever,
    return_source_documents=True
)

Corpus preparado con 100 reseñas
Corpus dividido en 100 chunks


Device set to use cpu


In [11]:

# --- 7) Consulta de prueba ---
query = "¿Hay algún comentario sobre le universo de Harry Potter?"
res = qa({"query": query})

print("Respuesta:")
print(res["result"])
print("\nFuentes:")
for d in res["source_documents"]:
    print(d.metadata)


Respuesta:
content: el universo mágico propuesto por Harry Potter es interesante, pese a algunos pasajes extensos verbatim verbatim verbatim

Fuentes:
{'sentimiento': 'neutro', 'titulo_pelicula': 'Harry Potter'}
{'sentimiento': 'neutro', 'titulo_pelicula': 'Harry Potter'}
{'titulo_pelicula': 'Harry Potter', 'sentimiento': 'negativo'}
{'titulo_pelicula': 'Harry Potter', 'sentimiento': 'negativo'}
{'sentimiento': 'negativo', 'titulo_pelicula': 'Harry Potter'}


### Semana 3: Interfaz Streamlit y Deployment

**Objetivos:**
- Aplicación Streamlit completa
- Sistema desplegado y accesible
- Documentación completa

**Tareas:**
- [ ] Crear aplicación Streamlit básica
- [ ] Implementar input de consultas
- [ ] Mostrar respuestas con formato
- [ ] Mostrar fuentes citadas (metadata de documentos)
- [ ] Agregar estados de carga (spinners)
- [ ] Implementar manejo de errores
- [ ] Opcional: Agregar historial de conversación (st.session_state)
- [ ] Opcional: Permitir cargar nuevos documentos desde la interfaz
- [ ] Configurar secrets (API keys) de forma segura
- [ ] Deployment según opción elegida
- [ ] Probar en el entorno desplegado
- [ ] Escribir README completo
- [ ] Documentar decisiones de diseño
- [ ] Testing con usuarios reales

**Estructura Streamlit recomendada:**
```python
import streamlit as st

st.set_page_config(page_title="Mi Sistema RAG", page_icon="🔍")

st.title("Sistema RAG: [Tu Dominio]")
st.markdown("Consultá información de [descripción del corpus]")

# Sidebar con información
with st.sidebar:
    st.header("Información")
    st.write("Corpus: X documentos")
    st.write("Modelo: [tu modelo]")
    st.markdown("---")
    st.subheader("Ejemplos de consultas:")
    st.write("- Ejemplo 1")
    st.write("- Ejemplo 2")

# Input principal
consulta = st.text_input(
    "Escribí tu consulta:",
    placeholder="Ejemplo: ¿Qué dice el documento sobre...?"
)

if st.button("Consultar", type="primary"):
    if not consulta:
        st.warning("Por favor, escribí una consulta.")
    else:
        with st.spinner("Buscando información relevante..."):
            try:
                resultado = qa_chain({"query": consulta})
                
                st.success("Consulta completada")
                
                # Respuesta
                st.subheader("Respuesta:")
                st.write(resultado["result"])
                
                # Fuentes
                st.subheader("Fuentes consultadas:")
                for i, doc in enumerate(resultado["source_documents"], 1):
                    with st.expander(f"Fuente {i}: {doc.metadata.get('source', 'Desconocido')}"):
                        st.write(doc.page_content[:300] + "...")
                        st.caption(f"Metadata: {doc.metadata}")
            
            except Exception as e:
                st.error(f"Error: {str(e)}")
                st.info("Intentá reformular tu consulta.")

# Footer
st.markdown("---")
st.caption("Trabajo Integrador N°2 - Procesamiento del Habla e Introducción a LLMs")
```

**Checkpoint final:**
- [ ] ¿La aplicación funciona en el entorno desplegado?
- [ ] ¿Alguien puede usarla siguiendo tu README?
- [ ] ¿Cumplís con TODOS los requisitos de aprobación?
- [ ] ¿El código tiene comentarios claros?
- [ ] ¿Las fuentes se muestran correctamente?
- [ ] ¿Probaste casos donde no hay respuesta en el corpus?

In [13]:
%%writefile app.py
import streamlit as st
import pandas as pd
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# --- Configuración de la página ---
st.set_page_config(page_title="Chatbot Harry Potter Reviews", page_icon="🪄")
st.title("🪄 Chatbot RAG: Reseñas de Harry Potter")
st.markdown("Consulta opiniones de usuarios sobre personajes y aspectos de las películas.")

# --- Sidebar ---
with st.sidebar:
    st.header("Información del corpus")
    st.write("Dataset: Harry Potter Reviews")
    st.write("Modelo: Flan-T5 base + Embeddings MiniLM-L6-v2")
    st.markdown("---")
    st.subheader("Ejemplos de consultas:")
    st.write("- ¿Qué comentarios negativos hay sobre Severus Snape?")
    st.write("- Opiniones de usuarios de España sobre Hermione Granger")
    st.write("- ¿Qué dicen los mayores de 40 años sobre Ron Weasley?")

# --- 1) Cargar dataset ---
@st.cache_data
def load_data():
    url = "https://raw.githubusercontent.com/michellzambranohereira/PLN/refs/heads/main/TP%20Final%20Integrador/dataset/rese%C3%B1as.csv"
    df = pd.read_csv(url)
    documentos = [
        Document(
            page_content=row["comment"],
            metadata={
                "user_id": row["user_id"],
                "user_sex": row["user_sex"],
                "user_age": row["user_age"],
                "user_country": row["user_country"],
                "rating": row["rating"],
                "favourite_character": row["favourite_character"],
                "date": row["date"]
            }
        )
        for _, row in df.iterrows()
    ]
    return documentos

documentos = load_data()

# --- 2) Preparar pipeline RAG ---
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
chunks = splitter.split_documents(documentos)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(chunks, embeddings, collection_name="hp_reviews_collection")
retriever = db.as_retriever(search_kwargs={"k": 8})

model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
hf_pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256, temperature=0)
llm = HuggingFacePipeline(pipeline=hf_pipe)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=retriever,
    return_source_documents=True
)

# --- 3) Interfaz principal ---
consulta = st.text_input("Escribí tu consulta:", placeholder="Ejemplo: ¿Qué comentarios negativos hay sobre Severus Snape?")
if st.button("Consultar", type="primary"):
    if not consulta:
        st.warning("Por favor, escribí una consulta.")
    else:
        with st.spinner("Buscando información relevante..."):
            try:
                resultado = qa_chain({"query": consulta})
                st.success("Consulta completada ✅")
                st.subheader("Respuesta:")
                st.write(resultado["result"])

                st.subheader("Fuentes consultadas:")
                for i, doc in enumerate(resultado["source_documents"], 1):
                    with st.expander(f"Fuente {i}"):
                        st.write(doc.page_content[:300] + "...")
                        st.caption(f"Metadata: {doc.metadata}")
            except Exception as e:
                st.error(f"Error: {str(e)}")
                st.info("Intentá reformular tu consulta.")



Overwriting app.py


In [14]:
!streamlit run app.py & npx localtunnel --port 8501


/bin/bash: line 1: streamlit: command not found
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) ^C


In [15]:
from google.colab import files
files.download("app.py")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Bloque 1 - Importación de librerías

In [ ]:
# Semana 3 - Importación de librerías

import pandas as pd
import numpy as np
import re
import string

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Modelos clásicos
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

# Métricas
from sklearn.metrics import classification_report, confusion_matrix

# Embeddings
from sentence_transformers import SentenceTransformer

# Clustering
from sklearn.cluster import KMeans

# Transformers para clasificación
from transformers import pipeline

# Interfaz
import gradio as gr


## Bloque 2: Cargar dataset

In [ ]:
df = pd.read_csv("reseñas.csv")

df.head()


,titulo_pelicula,reseña,sentimiento
0,Harry Potter,Cada libro de Harry Potter ofrece una evolució...,positivo
1,Harry Potter,La magia en Harry Potter está tan bien constru...,positivo
2,Harry Potter,La magia en Harry Potter está tan bien constru...,positivo
3,Harry Potter,La saga de Harry Potter me sorprendió por su m...,positivo
4,Harry Potter,Sentí que la historia de Harry Potter avanzaba...,negativo


##Bloque 3  — Preprocesamiento del texto

In [ ]:
def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r"http\S+", "", texto)
    texto = re.sub(r"@\w+", "", texto)
    texto = re.sub(r"#\w+", "", texto)
    texto = texto.translate(str.maketrans("", "", string.punctuation))
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

df["clean_text"] = df["reseña"].apply(limpiar_texto)
df.head()

,titulo_pelicula,reseña,sentimiento,clean_text
0,Harry Potter,Cada libro de Harry Potter ofrece una evolució...,positivo,cada libro de harry potter ofrece una evolució...
1,Harry Potter,La magia en Harry Potter está tan bien constru...,positivo,la magia en harry potter está tan bien constru...
2,Harry Potter,La magia en Harry Potter está tan bien constru...,positivo,la magia en harry potter está tan bien constru...
3,Harry Potter,La saga de Harry Potter me sorprendió por su m...,positivo,la saga de harry potter me sorprendió por su m...
4,Harry Potter,Sentí que la historia de Harry Potter avanzaba...,negativo,sentí que la historia de harry potter avanzaba...


## Bloque 4 - Vectorización TF-IDF

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df["clean_text"])
y = df["sentimiento"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


## Bloque 5 — Modelos Clásicos

### 5.1 Naive Bayes

In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
pred_nb = nb.predict(X_test)

print("NAIVE BAYES\n")
print(classification_report(y_test, pred_nb))


NAIVE BAYES

              precision    recall  f1-score   support

    negativo       1.00      1.00      1.00         6
      neutro       1.00      1.00      1.00         6
    positivo       1.00      1.00      1.00         8

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



### 5.2 Logistic Regression

In [ ]:
logreg = LogisticRegression(max_iter=300)
logreg.fit(X_train, y_train)
pred_lr = logreg.predict(X_test)

print("LOGISTIC REGRESSION\n")
print(classification_report(y_test, pred_lr))


LOGISTIC REGRESSION

              precision    recall  f1-score   support

    negativo       1.00      1.00      1.00         6
      neutro       1.00      1.00      1.00         6
    positivo       1.00      1.00      1.00         8

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



### 5.3 SVM

In [ ]:
svm = LinearSVC()
svm.fit(X_train, y_train)
pred_svm = svm.predict(X_test)

print("SVM\n")
print(classification_report(y_test, pred_svm))


SVM

              precision    recall  f1-score   support

    negativo       1.00      1.00      1.00         6
      neutro       1.00      1.00      1.00         6
    positivo       1.00      1.00      1.00         8

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



### 5.4 Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=200)
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)

print("RANDOM FOREST\n")
print(classification_report(y_test, pred_rf))


RANDOM FOREST

              precision    recall  f1-score   support

    negativo       1.00      1.00      1.00         6
      neutro       1.00      1.00      1.00         6
    positivo       1.00      1.00      1.00         8

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



## BLOQUE 6 — Embeddings + Clustering

In [ ]:
model_emb = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = model_emb.encode(df["clean_text"].tolist(), show_progress_bar=True)

# KMeans
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
df["cluster"] = kmeans.fit_predict(embeddings)

df[["clean_text", "cluster"]].head()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

,clean_text,cluster
0,cada libro de harry potter ofrece una evolució...,0
1,la magia en harry potter está tan bien constru...,2
2,la magia en harry potter está tan bien constru...,2
3,la saga de harry potter me sorprendió por su m...,0
4,sentí que la historia de harry potter avanzaba...,2


## BLOQUE 7 — Modelo con Transformers (Sentimiento)

In [ ]:
sentiment_model = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-xlm-roberta-base-sentiment"
)

def clasificar_transformer(texto):
    resultado = sentiment_model(texto)[0]
    return f"{resultado['label']} ({round(resultado['score'],2)})"

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


## BLOQUE 8 — Interfaz con Gradio

In [ ]:
import gradio as gr

def predecir_sentimiento(texto):
    # Modelo clásico + transformer, podés elegir qué devolver
    pred_svm = svm.predict(tfidf.transform([texto]))[0]
    pred_trans = clasificar_transformer(texto)

    return f"""
    🧠 SVM: {pred_svm}
    🤖 Transformer: {pred_trans}
    """

# Interfaz
gr.Interface(
    fn=predecir_sentimiento,
    inputs=gr.Textbox(label="Ingrese un texto"),
    outputs=gr.Textbox(label="Resultado"),
    title="Clasificador de Sentimiento — Semana 3"
).launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8157eaf175f91c8b1c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
